In [1]:
"""Training for Grounded SAM.
"""
import os
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
from torchvision import transforms
from PIL import Image
from transformers import SamProcessor

from linear_probe import LinearProbe
from model import load_model, preprocess_sam, preprocess_groundingdino_img
from groundingdino.util.misc import nested_tensor_from_tensor_list
from utils import get_bounding_box, SAMDataset

from model import myGroundingDino, myBiomedCLIP, mySAM
import dataset_mimic
import dataset_pascal

In [ ]:
def train_adaptation():
    """Training script for adaptation loss only (to be combined with segmentation objective).
    """
    # Load data
    mimic_dataloader = dataset_mimic.get_dataloader(batch_size=16, tensor=True)

    # Load model
    my_groundingdino = MyGroundingDino()
    my_sam = 